In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [3]:
PATH = "pdf/Livro de Arquitetura e organização de computadores.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [4]:
text_split = []
metadata=[]
chunk_size = 432
chunk_overlap = 8

for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})
embeddings_db = embeddings.encode(text_split)

In [5]:
for i, d in enumerate(text_split):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadata[i]
    )

In [ ]:
while True:

    query = []

    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=10,
            where={"name": "book-langchain"}
        )

    data = results['documents'][0]
    distance = results['distances'] 
    print(distance)
    
    for k in range(10):
        if distance[0][k] < 0.74:
            query.append(data[k])

    prompt_template = f"O seu único conhecimento é sobre {query}. Agora, responda {prompt}"
    print(query)

    if len(query) == 0:
            prompt_template = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    
    output = ollama.generate(
            model="llama2",
            prompt=prompt_template
            )
    print("\n")
    output = output['response']
    print(output)
    print("\n")
    

Pergunte algo o que é cpu?


[[0.79204261302948, 0.8166285157203674, 0.8187282681465149, 0.8256652355194092, 0.8310142755508423, 0.8341817855834961, 0.8365865349769592, 0.8421618342399597, 0.8493126630783081, 0.8627681732177734]]
[]



Ah, entendo! Em que especifico você precisa de ajuda? Eu sou apenas um assistente de chatbot com acesso limitado às informações, então posso dizer que não tenho conhecimentos sobre o assunto da pergunta. Por favor, forneça mais detalhes ou informações específicas que você precisa de ajuda com.




Pergunte algo o que é memória cache?


[[0.34884095191955566, 0.41957995295524597, 0.44657766819000244, 0.4675009846687317, 0.4697117209434509, 0.47685664892196655, 0.4823729991912842, 0.49521246552467346, 0.4972468316555023, 0.5186479091644287]]
['ria CaChe\nA memória cache é desenvolvida para combinar o tempo de acesso de memórias de alto custo e alta ve-\nlocidade com as memórias de menor velocidade, maior tamanho e mais baixo custo. O conceito é ilustrado na Figura 4.3a. Existe uma memória principal relativamente grande e lenta junto com a memória cache, menor e \nmais rápida. A cache contém uma cópia de partes da memória principal. Quando o processador tenta ler uma \npalavra da memória,', 'mória de um nível.\nO mecanismo de cache da memória principal faz parte da arquitetura do computador, implementada \nno hardware e normalmente invisível ao sistema operacional. Existem dois outros casos de uma técnica \nde memória de dois níveis que também exploram a localidade e que são, pelo menos parcialmente, imple-\nmentadas no

Pergunte algo o que é memória swap?


[[0.6941795349121094, 0.7467305660247803, 0.7506506443023682, 0.7635656595230103, 0.7959352731704712, 0.8045865297317505, 0.8366129398345947, 0.8367538452148438, 0.8441327810287476, 0.844537615776062]]
['ar o máximo possível de processos na memória.\ntroca de processos na memória (swapping)\nVoltando à Figura 8.11, discutimos três tipos de filas: a fila de longo prazo de solicitações de novos proces-\nsos, a fila de curto prazo de processos prontos para usar o processador e as diversas filas de E/S dos processos que não estão prontos para usar o processador. Lembre-se de que o motivo para esse mecanismo elaborado é Figura 8.11\nDiagrama de filas do']



 Memória swap é um tipo de memória virtualextensão que permite ao sistema operacional intercambiar (swapping) processos entre a memória RAM e a disk storage when the system is running low on physical memory resources. This mechanism allows the system to temporarily transfer less frequently used processes or data from the RAM to the hard

Pergunte algo o que é computação em nuvem?


[[0.36352550983428955, 0.6337176561355591, 0.6677056550979614, 0.6880044341087341, 0.7073640823364258, 0.7263635993003845, 0.7334384918212891, 0.7347819805145264, 0.7410671710968018, 0.7425028085708618]]
['ocontrolador.\n1.7 COMPutaçãO eM nuveM\nEmbora os conceitos gerais para computação em nuvem remontem à década de 1950, os serviços de com-\nputação em nuvem tornaram-se disponíveis pela primeira vez nos anos 2000, direcionados sobretudo a grandes empresas. Desde então, a computação em nuvem expandiu-se para pequenos e médios negócios e mais recen-temente tem chegado aos consumidores. O iCloud da Apple foi o primeiro a ser lançado, em 2012, e obtev', 'tura\nAo se descreverem computadores, é comum se fazer uma distinção entre arquitetura de computadores e \norganização de computadores . Embora seja difícil estabelecer definições precisas  para tais termos, existe um \nconsenso a respeito das áreas que abrangem cada um. (Por exemplo, veja VRANESIC; THURBER, 1980, \nSIEWIOREK; BELL; NEWE